# Strands Agents セッション間でのメモリの永続化

この例では、Strands Agents の異なるセッション間でメモリを永続化する方法を学びます。

`duckduckgo` 検索 API を使用して Web 検索を行うエージェントをユースケースとして使用します。

このノートブックでは、次のことを行います。
- メモリ駆動型 Strands エージェントの機能について調べる。
- メモリの保存、取得、一覧表示の方法を学ぶ。
- エージェント経由で Web 検索を実行する方法を理解する。
- 対話型ループでエージェントと対話する。

### 使用例

メモリの保存:
```
私はコーヒーよりも紅茶が好きだと覚えておく。
```

メモリの取得:
```
私は何を飲むのが好きか？
```

すべての記憶を一覧表示する:
```
私について覚えていることをすべて見せてください
```

### 記憶の使用に関するヒント

- エージェントに情報を記憶するよう依頼する際は、明確に指示してください
- 関連する記憶を検索するには、具体的なクエリを使用してください
- 記憶の永続化により、より自然で文脈に沿った会話が可能になります

## セットアップと前提条件

### 前提条件
* Python 3.10 以上
* 環境に設定された AWS アカウントと AWS 認証情報
* Amazon Bedrock で Anthropic Claude 3.7 が有効化されていること
* Amazon Bedrock ナレッジベース、Amazon S3 バケット、Amazon DynamoDB を作成する権限を持つ IAM ロール

Strands エージェントに必要なパッケージをインストールしましょう

In [ ]:
# 必要なパッケージのインストール
!pip install -r requirements.txt

In [ ]:
# 必要なライブラリのインポート
import os
from strands import Agent, tool
from strands_tools import mem0_memory
from duckduckgo_search import DDGS
from duckduckgo_search.exceptions import DuckDuckGoSearchException, RatelimitException

## Mem0 構成

### メモリバックエンドオプション

Mem0 メモリツールは、3 つの異なるバックエンド構成をサポートしています。

1. **[OpenSearch](https://aws.amazon.com/opensearch-service/features/serverless/)** (AWS 環境に推奨):
- AWS 認証情報と OpenSearch 構成が必要です。
- `OPENSEARCH_HOST` と、オプションで `AWS_REGION` を設定します (デフォルトは us-west-2)。

2. **[FAISS]((https://faiss.ai/index.html))** (ローカル開発のデフォルト):
- ローカルベクターストアのバックエンドとして FAISS を使用します。
- ローカルベクターストレージには faiss-cpu パッケージが必要です。
- 追加の構成は不要です。

3. **Mem0 プラットフォーム**:
- メモリ管理に [Mem0 プラットフォーム API](https://docs.mem0.ai/platform/quickstart) を使用します。
- Mem0 API キーが必要です。 : 環境変数の `MEM0_API_KEY`

### 環境設定

| 環境変数 | 説明 | デフォルト | 必須 |
|---------------------|------------|----------|--------------|
| OPENSEARCH_HOST | OpenSearch サーバーレスホストの URL | なし | OpenSearch |
| AWS_REGION | OpenSearch の AWS リージョン | us-west-2 | OpenSearch |
| MEM0_API_KEY | Mem0 プラットフォームの API キー | なし | Mem0 プラットフォーム |
| DEV | 開発モードを有効にする | false | すべてのモード |

このラボでは、メモリ管理のバックエンドとして以下の 2 つのオプションを使用できます。
### オプション 1. [Opensearch Serverless](https://aws.amazon.com/opensearch-service/features/serverless/)

以下が AOSS のセットアップアーキテクチャです。

<div style="text-align:left">
<img src="images/arch_AOSS.png" width="65%" />
</div>



In [ ]:
# OpenSearch ホストを手動で定義
#os.environ["OPENSEARCH_HOST"] = "<your-opensearch-host>.<region>.aoss.amazonaws.com"

In [ ]:
# または - スクリプトを実行して AWS アカウントに OpenSearch Serverless リソースを作成
!bash prereqs/deploy_OSS.sh

In [ ]:
# オプション1: OpenSearch Serverless
from dotenv import load_dotenv
load_dotenv() # OpenSearch 環境変数を取得する

### オプション 2 [Mem0 プラットフォーム](https://docs.mem0.ai/platform):

#### [注]: Opensearch Serverless オプションを既にデプロイしている場合は、この手順は不要です。

代わりに、[こちら](https://docs.mem0.ai/platform/quickstart#2-api-key-setup) の手順に従って Mem0 API キーを作成し、それを環境変数 **MEM0_API_KEY** として追加することもできます。
Mem0 プラットフォームのセットアップのアーキテクチャは以下のとおりです。

<div style="text-align:left">
<img src="images/arch_mem0.png" width="65%" />
</div>

エージェントの機能を有効にするには、AWS 認証情報と OpenSearch / Mem0 プラットフォームの環境変数を設定する必要があります。これらの変数は、メモリの保存と取得に使用されます。

In [ ]:
# オプション2: Mem0 APIキー

#os.environ["MEM0_API_KEY"] = "<your-mem0-api-key>"

## システムプロンプトの定義

`SYSTEM_PROMPT` 変数は、メモリエージェントの動作と機能を定義します。このプロンプトは、エージェントが保存された記憶に基づいてパーソナライズされた応答を提供し、必要に応じて Web 検索を実行するようにガイドします。

In [ ]:
# メモリ操作のためのフォーカスされたシステムプロンプトを定義
SYSTEM_PROMPT = """あなたはユーザーにとって頼りになるパーソナルアシスタントです。あなたの仕事は、ユーザーの履歴に基づいてパーソナライズされた応答を提供することで、ユーザーを支援することです。
機能：
  - mem0_memory ツール（action="store"）を使用して情報を保存できます。
  - mem0_memory ツール（action="retrieve"）を使用して、関連する記憶を取得できます。
  - duckduckgo_search を使用して、Web 上で情報を検索できます。
重要なルール：
  - 応答は会話的で自然なものにしてください。
  - ユーザーに応答する前に必ず記憶を取得し、それに基づいて応答してください。
  - 新しいユーザー情報とユーザーの設定は、mem0_memory に保存してください。
  - 関連のある情報のみを共有してください。
  - 情報がない場合には、丁寧にその旨を伝えてください。
"""

## Web検索ツールの定義

[Duckduckgo Search API](https://github.com/deedy5/duckduckgo_search) 関数を使用した `websearch` ツールにより、エージェントはWeb検索を実行できます。この関数は例外を処理し、検索結果または適切なエラーメッセージを返します。

In [ ]:
@tool
def websearch(
    keywords: str,
    region: str = "us-en",
    max_results: int | None = None,
) -> str:
    """ウェブを検索して最新情報を取得します。
       引数:
         keywords (str): 検索クエリキーワード。
         region (str): 検索地域: wt-wt、us-en、uk-en、ru-ru など。
         max_results (int | None): 返される結果の最大件数。
       戻り値:
        検索結果を含む辞書のリスト。
    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "No results found."
    except RatelimitException:
        return "RatelimitException: Please try again after a short delay."
    except DuckDuckGoSearchException as d:
        return f"DuckDuckGoSearchException: {d}"
    except Exception as e:
        return f"Exception: {e}"

## メモリエージェントの作成

定義したツールとシステムプロンプトを使用して、メモリに重点を置いたエージェントを初期化します。Strandsエージェントは以下の機能を備えています。
1. コンテキストに基づいてメモリを保存および取得します。メモリを使用することで、よりパーソナライズされ、コンテキストに基づいたAIインタラクションを実現します。
2. DuckDuckGoを使用してWeb検索を実行し、最新情報を提供します。

In [ ]:
# メモリとウェブ検索ツールを備えたエージェントを作成
USER_ID = "new_user" # 実際のユーザーIDに置き換える

memory_agent = Agent(
    system_prompt=SYSTEM_PROMPT,
    tools=[mem0_memory, websearch],
)

## メモリ操作のデモンストレーション

以下の例では、メモリエージェントを使用してメモリを保存、取得、および一覧表示する方法を示します。

- **保存**: 重要な情報を後で取得できるように保存する
- ユーザーの設定を保存する
- 重要な事実を記憶する
- 会話のコンテキストを維持する

- **取得**: クエリに基づいて関連するメモリにアクセスする
- 以前に保存した情報を検索する
- ユーザーの履歴に基づいてパーソナライズされた応答を提供する

- **一覧**: 保存されているすべてのメモリを表示する
- 保持されている情報を確認する
- 保存されているメモリを監査する

In [ ]:
# 初期の記憶を保存して取り出す
memory_agent.tool.mem0_memory(
    action="store", content=f"ユーザー名は {USER_ID} です。", user_id=USER_ID
)
memory_agent.tool.mem0_memory(
    action="store", 
    content="私はコーヒーよりもお茶を飲むのが好きです。", 
    user_id=USER_ID
)

In [ ]:
# Retrieve memories
retrieved_memories = memory_agent.tool.mem0_memory(
    action="retrieve", query="ユーザーの名前は何ですか?", user_id=USER_ID
)
print("Retrieved Memories:", retrieved_memories)

In [ ]:
# Retrieve memories about preferences
memory_agent.tool.mem0_memory(
    action="retrieve",
    query="私の飲み物の好みは何ですか?",
    user_id=USER_ID
)

In [ ]:
# Ask the agent a question
response = memory_agent("今日ニューヨークで何が起きているのでしょうか？")
print(response)

In [ ]:

# List all stored memories
print("All Stored Memories:")
all_memories = memory_agent.tool.mem0_memory(
    action="list", user_id=USER_ID
)

## 対話型エージェントの使用

最後に、ユーザーがメモリエージェントと対話するための対話型ループを提供します。ユーザーは、新しいメモリを保存したり、既存のメモリを取得したり、保存されているすべてのメモリを一覧表示したりできます。

対話型の使用をテストするには：
1. 要件をインストールします：`pip install -r requirements.txt`
1. Pythonファイル `personal_agent_with_memory.py` を実行します。

## まとめ

このノートブックでは、Strandsフレームワークを用いて記憶機能を備えたパーソナルエージェントを作成する方法を説明します。エージェントは以下のことが可能です。

1. ユーザーに関する情報を保存する
2. コンテキストに基づいて関連する記憶を検索する
3. Webで追加情報を検索する
4. パーソナライズされた応答を提供する

これらの機能を組み合わせることで、エージェントは会話全体を通してコンテキストを維持し、時間の経過とともによりパーソナライズされた支援を提供できるようになります。

### クリーンアップ

Opensearch Serverless リソースをクリーンアップするには、この bash スクリプトを実行します。「MEM0_PLATFORM_API」を使用した場合は、このスクリプトを実行する必要はありません。

In [ ]:
!sh prereqs/cleanup_OSS.sh